# NanoPyx "Codeless" Jupyter Notebook

This notebook allows you to perform channel registration of 2D multi-channel image stacks

To use this notebook you don't need to interact with any code, just run cells in order and a graphical user interface will pop-up showcasing the parameters for each step.
When running this notebook on Colab, it will automatically prompt you to connect to your google Drive, from where you can select the input data.
To save the output of this notebook, make sure to select the "save output" option. Output will be saved on the same directory as the loaded image. If using an example dataset it will be saved on the path Python is currently running (if locally running the notebook) or on Colab files (if running on Colab, files button on the left bar).
If you found this work useful for your research please consider citing our [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1)  and out Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398)

**Channel Registration** Romain F Laine et al 2019 J. Phys. D: Appl. Phys. 52 163001




In [ ]:
#@title Install NanoPyx, import necessary libraries and connect to Google Drive
!pip install -q "nanopyx[jupyter]"
import io
import os
import sys
import skimage
import nanopyx
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from IPython.display import display, clear_output
from matplotlib import pyplot as plt

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !pip install -q ipycanvas==0.11.0
    from google.colab import output
    output.enable_custom_widget_manager()
    from google.colab import drive
    drive.mount('/content/drive')

cwd = os.getcwd()
image_folder = "datasets"
image_files = []
EDM = ExampleDataManager()
example_datasets = EDM.list_datasets()

_path = os.path.join("..", image_folder)
if os.path.exists(_path):
    image_files += find_files(_path, ".tif")
if os.path.exists(image_folder):
    image_files += find_files(image_folder, ".tif")
image_files += ["Example dataset: "+dataset for dataset in example_datasets]




In [ ]:
#@title Create image loader GUI
# Create a GUI

gui_data = EasyGui("Data Loader")

def on_button_load_data_clicked(b):
    clear_output()
    gui_data.show()
    global dataset_original
    # disable button
    gui_data["load_data"].disabled = True
    gui_data["load_data"].description = "Loading..."

    if gui_data["data_source"].value.startswith("Example dataset: "):
        dataset_name = gui_data["data_source"].value.replace(
            "Example dataset: ", "")
        dataset_original = EDM.get_ZipTiffIterator(dataset_name, as_ndarray=True)
        display(stackview.slice(dataset_original, continuous_update=True,
                                colormap=gui_data["cmaps"].value))
    else:
        dataset_original = skimage.io.imread(gui_data["data_source"].value)
        display(stackview.slice(dataset_original, continuous_update=True,
                                colormap=gui_data["cmaps"].value))
    
    # enable button
    gui_data["load_data"].disabled = False
    gui_data["load_data"].description = "Load data"
    gui_data.save_settings()

gui_data.add_label("Select data to use:")
gui_data.add_dropdown("data_source", options=image_files,
                 value="Example dataset: "+example_datasets[4], remember_value=True)
gui_data.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_data.add_button("load_data", description="Load data")
gui_data["load_data"].on_click(on_button_load_data_clicked)
gui_data.show()




# Channel Registration Parameters:  

Requires an image stack with shape: (channel, rows, columns).  

- **Reference Channel:** Which channel to be used as reference.  
- **Max Expected Shift:** Maximum amount of expected shift between channels, in pixels.  
- **Blocks per Axis:** As channel misalignmnet is not always homogeneous across the field of view, shift can be calculated for individual blocks of the field of view. This parameters sets how many blocks are created along both axis.  
- **Minimum Similarity:** Since smaller blocks may lead to shift calculation in areas of the image without any cells, minimum similarity can be used to define the minimum Pearson's Correlation Coefficient, between two blocks of different channels, required to use the calculated shifts as part of the registration. 




In [ ]:
#@title Create channel registration GUI
from nanopyx.methods import channel_registration

gui_reg = EasyGui("Channel Registration")

def on_button_register(b):
    clear_output()
    gui_reg.show()
    ref_channel = gui_reg["ref"].value
    max_shift = gui_reg["max"].value
    n_blocks = gui_reg["blocks"].value
    min_sim = gui_reg["min_sim"].value
    global dataset_registered
    gui_reg["register"].disabled = True
    gui_reg["register"].description = "Aligning..."
    dataset_registered = channel_registration.estimate_channel_registration(dataset_original,
                                                                            ref_channel,
                                                                            max_shift,
                                                                            n_blocks,
                                                                            min_sim,
                                                                            save_translation_masks=False,
                                                                            save_ccms=False,
                                                                            apply=True)
    if gui_reg["save"].value:
        if own_data:
            path = gui_data["upload"].selected_path
            name = gui_data["upload"].selected_filename.split(".")[0]
            tiff.imwrite(path + os.sep + name + "_registered.tif", dataset_registered)
        else:
            name = gui_data["data_source"].value.replace("Example dataset: ", "")
            tiff.imwrite(name + "_registered.tif", dataset_registered)
    gui_reg["register"].disabled = False
    gui_reg["register"].description = "Align"
    display(stackview.slice(dataset_registered, colormap=gui_reg["cmaps"].value,
                            continuous_update=True))

gui_reg.add_label("Channel Registration parameters:")
gui_reg.add_int_slider("ref", description="Reference channel", min=0, max=dataset_original.shape[0]-1, value=0)
gui_reg.add_int_slider("max", description="Max expected drift", min=0, max=1000, value=10)
gui_reg.add_int_slider("blocks", description="Blocks per axis", min=1, max=10, value=5)
gui_reg.add_float_slider("min_sim", description="Minimum similarity", min=0, max=1, value=0.5, step=0.1)
gui_reg.add_dropdown("cmaps", description="Colormap:",
                     options=sorted(list(mpl.colormaps)),
                     value="viridis", remember_value=True)
gui_reg.add_checkbox("save", description="Save Output", value=True)
gui_reg.add_button("register", description="Register")
gui_reg["register"].on_click(on_button_register)
gui_reg.show()

